In [1]:
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel


def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large')


In [2]:
def get_embedding(text: str):
    batch_dict = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**batch_dict)
    embedding = F.normalize(average_pool(outputs.last_hidden_state, batch_dict['attention_mask']))
    return embedding

In [3]:
output = float(F.cosine_similarity(get_embedding("I lost my car keys"), get_embedding("I lost my safe deposit keys")))
output

0.9097751379013062